[Follow this Conceptual Introduction by Betancourt (2018)](https://arxiv.org/pdf/1701.02434.pdf)

> The geometry of high-dimensional probability distributions...frustrates efficient statistical computing.

Given a target probability distributions, we would like to compute expectations with respect to the distribution. Bayesian inference: get info from a posterior distribution over model configuration space.

Consider smooth, $D$-dimensional sample space $Q$ upon which we have our target distribution $\pi$, and expectations $\mathbb{E}_{\pi}\left[f\right]$. These expectations are integrals over parameter space: $$ \mathbb{E}_{\pi}\left[f\right] = \int_Q dq \, \pi\left(q\right) f\left(q\right) $$ These expectations must be invariant under change of parametrization of the parameter space $Q \mapsto Q'$.

We must approximate these integrals.

Key: evaluating the target density $\pi$ and functions $f$ in irrelevant regions of parameter space --- regions which contribute very little to the expectation --- wastes a great deal of resources. We want to focus only on "important regions."

For now, ignore the changes in the function $f$; we may wish to use different functions, and it is not sensible to focus on any one particular form of function $f$. Instead, we will focus mainly on the target density.

Methods like MLE and Laplace approximation focus mainly around the mode of the distribution, but this region generally does not occupy a large volume in parameter space. We need to consider both the density and the local volume in order to understand what dominates the expectations. Note that
* regions near the mode have high density, but small volume;
* regions far from the mode have large volume, but lower density.

The *typical set* is between these extremes. It becomes more singular as dimension increases (*concentration of measure*, I think this is the "curse of dimensionality").

Key: we can accurately estimate expectations by averaging over the typical set instead of all parameter space. How can we identify the typical set?

### Markov Chain Monte Carlo

MCMC is one approach for stochastically exploring the typical set. A Markov chain will eventually explore the typical set of **any** target distribution. Can we do it in real time?

A Markov chain is a sequence of points which are moved between according to a random map called a Markov transition: conditional probability $T\left(q'\middle|q\right)$.

General Markov chains wander. Instead, we want it to preserve the target distribution, i.e., $$ \pi\left(q\right) = \int_Q dq' \, \pi\left(q'\right) T\left(q\middle|q'\right) $$ Why? Because then the Markov transition distribution will tend toward the typical set.

If we repeat this process enough, the samples $q_0, q_1,\ldots,q_n$ generated by the chain come to quantify the typical set. We can estimate expectations over the typical set by averaging the target function over this trace: $$ \hat{f}_n = \frac{1}{n} \sum_{i=1}^n f\left(q_i\right) $$ where we have that $$ \lim_{n\rightarrow \infty} \hat{f}_n = \mathbb{E}_{\pi}\left[f\right] $$

All this requires that the Markov transition $T$ is compatible with the structure of the target distribution $\pi$. Markov transitions often have difficulty with high-curvature regions.

Given a transition which correctly targets the target distribution, we can proceed with MCMC to quantify the typical set. But how do we come up with such a transition distribution?

#### Metropolis-Hastings

The MH algorithm is a method for automatically constructing appropriate transitions. It operates in a two-step process:
1. proposal: a stochastic perturbation of the initial step is generated;
2. correction: the proposal is accepted or rejected based on distance from target step.


Let $Q\left(q'\middle|q\right)$ be the proposal-generating probability density. The acceptance probability is then $$ \min \left(1 , \frac{Q\left(q\middle|q'\right)}{Q\left(q'\middle|q\right)} \frac{\pi\left(q'\right)}{\pi\left(q\right)}\right) $$ A common choice for proposal mechanisms is $$ Q\left(q\middle|q'\right) = \mathcal{N}\left(q'\middle|q,\Sigma\right) $$ Note that when the proposal distribution is symmetric under argument interchange, the $Q$ factors cancel.

Bad news: this is slow, especially in high-dimensional parameter spaces: there are too many possible dimensions to consider moving in.

### Hamiltonian Monte Carlo

HMC explores the target distribution "by carefully exploting the differential structure of the target probability density."

By differential structure we of course refer to the gradient of the target probability distribution, since this informs us of the local structure. In particular, gradient defines a vector field in parameter space.

Unfortunately, as we know from calculus, the gradient aims for the mode of the distribution, not the typical set. We need to use further geometric constraints. Differential geometry provides the solution!

Rather than build up this theory from the ground up, HMC is developed by an analogy to physics.

>If we place a satellite at rest out in space it will fall in the gravitational field and crash into the surface of the planet, just as naive gradient-driven trajectories crash into the mode.

> We can maintain a stable orbit by endowing our satellite with enough *momentum* to counteract the gravitational attraction.

We need to augment our system with auxiliary momentum parameters in order to align the "gradient" vector field with the typical set.

>They need to be endowed with a probabilistic structure that ensures conservative dynamics.

This is the goal of HMC.

Conservative physical systems: volumes (of position-momentum phase space) are preserved. A compression in position is compensated by an expansion in momentum.

Given dimension $n$ of our target parameter space, say $q_n$, we introduce momentum params $p_n$ to obtain a 2D parameter space. These momenta are dual to the target params, which keeps phase space volume invariant.

The **canonical distribution** is the joint prob dist over phase space, $$ \pi\left(q,p\right) = \pi\left(p\middle|q\right)\pi\left(q\right) $$ which requires us to choose a conditional dist over momentum (the $\pi\left(p\middle|q\right)$).

Note that this marginalizes to the target distribution. Because of this, the typical set in phase space (2D) "projects" down to the typical set in the target space.

Because of $q$ and $p$ being dual, the canonical dist $\pi\left(q,p\right)$ is indept of param, and we can write $$ \pi\left(q,p\right) = e^{-H\left(q,p\right)}$$ where $H$ is an invariant Hamiltonian function. This is the energy at a point in phase space, in physics. Here, it "captures the invariant probabilistic structure of the phase space distribution and...the geometry of its typical set."

$$ \begin{align} H\left(q,p\right) & = - \log \pi\left(q,p\right) \\ & = -\log \pi\left(p\middle|q\right) -\log \pi\left(q\right) \\ & \equiv  K\left(p,q\right) + V\left(q\right) \end{align} $$ where the two terms are the kinetic and potential energy, respectively. Potential is sepcified by the target dist, and kinetic energy is unconstrained, "must be specified by the implementation."

Applying Hamilton's equations gives us a vector field: $$ \begin{align} \frac{dq}{dt} & = + \frac{\partial H}{\partial p} = \frac{\partial K}{\partial p} \\ \frac{dp}{dt} & = - \frac{\partial H}{\partial q} = -\frac{\partial K}{\partial q}-\frac{\partial V}{\partial q} \\ \end{align} $$

Note that $\frac{\partial V}{\partial q} = \nabla \log \pi\left(q\right) $.

We can integrate (follow) this vector field to generate trajectories $\phi_t\left(q,p\right)$ which move rapidly through the phase space and along the typical set. We can then project these back down onto the target parameter space.

How to actually assign a momentum to a target space coordinate, in practice? **Sample from the conditional distribution over the momentum,** $$ p\sim \pi\left(p\middle|q\right) $$

Upon implementation, we need to be concerned with 

1. choice of kinetic energy/the distribution $p\left(p\middle|q\right)$;
2. Hamilton's equations solvers

# Implementation

Change-point example:  $$ \begin{align} n & \sim \text{Uniform}\left(1,2,\ldots,N\right) \\ \lambda_i & \sim \text{Gamma}\left(\lambda_i;a,b,\right) \\ x_i & \sim \left\{  \begin{array}{rc} \text{Poisson}\left(x_i;\lambda_1\right) & 1\leq i \leq n \\ \text{Poisson}\left(x_i;\lambda_2\right) & n < i \leq N     \end{array}    \right.  \end{align} $$